# git2net analysis for the networkx repository

First we import the required packages

In [132]:
import pygit2 as git2
import os
import shutil
import git2net
import pathpy as pp
import sqlite3
import pandas as pd
from datetime import date, datetime, timedelta
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.dates import date2num
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import json 
import math
import copy
import networkx as nx
import random
from statistics import mean, stdev, StatisticsError
import gambit
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Bipartite networks
### Converting bipartite network into WOMP (weighted one-mode projection)

In [98]:
def collab_network(sqlite_db_file, git_repo_dir, t, node_info, min_date, max_date, file_base=True):
    if file_base:
        start = int(datetime.timestamp(min_date))
        end = int(datetime.timestamp(max_date))

        n = pp.Network.from_temporal_network(t, min_time=start, max_time=end)
        
        n = remove_duplicate_filenames(n, git_repo_dir)
        n = add_weights(n, node_info)
        
        new_n = copy.deepcopy(n)

        for node1 in n.nodes:
            if node_info['class'][node1] == 'author':
                for node2 in n.nodes:
                    if node_info['class'][node2] == 'author' and node1 != node2:
                        intersect = n.successors[node1].intersection(n.successors[node2])
                        w = 0
                        for file in intersect:
                            #print((node1, file), ':', n.edges[(node1, file)])
                            w = w + n.edges[(node1, file)]['wijLR']
                        if w > 0:
                            new_n.add_edge(node1, node2, weight=w)

        for node in n.nodes:
            if node_info['class'][node] == 'file':
                new_n.remove_node(node)

        return new_n
    else:
        n, node_info, edge_info = git2net.get_coauthorship_network(sqlite_db_file, time_from=min_date, time_to=max_date)
        return n

In [99]:
def remove_duplicate_filenames(n, git_repo_dir):
    all_files = []
    for path, subdirs, files in os.walk(git_repo_dir):
        for name in files:
            all_files.append(name)

    duplicates = [file for file in all_files if all_files.count(file) > 1]
    remove_files = set(duplicates)

    new_n = copy.deepcopy(n)
    for node in new_n.nodes:
        if node in remove_files:
            n.remove_node(node)
    del new_n
    return n

In [100]:
def duplicate_filenames(git_repo_dir):
    all_files = []
    for path, subdirs, files in os.walk(git_repo_dir):
        for name in files:
            all_files.append(name)

    duplicates = [file for file in all_files if all_files.count(file) > 1]
    remove_files = set(duplicates)
    print(len(remove_files))
    
    #keep_nodes = [f for f in all_files if f not in remove_files]
    #print(len(keep_nodes))
    #print(keep_nodes)
    
    #t = t.filter_nodes(keep_nodes)

    #new_n = copy.deepcopy(n)
    #for node in new_n.nodes:
    #    if node in remove_files:
    #        n.remove_node(node)
    #del new_n
    return remove_files

In [101]:
def add_weights(n, node_info):
    for node in n.nodes:
        if node_info['class'][node] == 'author':
            s = 0
            for successor in n.successors[node]:
                for key in n.edges:
                    if key[0] == node and key[1] == successor:
                        s = s + n.edges[key]['weight']

        n.nodes[node]['WiL'] = s
    for edge in n.edges:
        n.edges[edge]['wijLR'] = n.edges[edge]['weight'] / n.nodes[edge[0]]['WiL']
    
    return n

In [102]:
def collab_network_jaccard(sqlite_db_file, git_repo_dir, t, node_info, min_date, max_date, duplicate_files, file_base=True, is_temp=True):
    if file_base:
        start = int(datetime.timestamp(min_date))
        end = int(datetime.timestamp(max_date))
        
        #print('1. From temporal network...')
        if is_temp:
            n = pp.Network.from_temporal_network(t, min_time=start, max_time=end)
        else:
            n = t
        
        #print('2. Remove duplicate files... SKIPPED')
        #n = remove_duplicate_filenames(n, git_repo_dir)
        
        #print('3. New empty network')
        new_n = pp.Network(directed=False)
        
        #print('4. Add authors to network')
        nn = copy.deepcopy(n.nodes)
        for node in nn:
            if node_info['class'][node] != 'file':
                new_n.add_node(node)
            elif node in duplicate_files:
                n.remove_node(node)
        
        #new_n = new_n.to_undirected()  
        
        #print('5. Jaccard loop')
        visited_nodes = []
        for node1 in n.nodes:
            if node_info['class'][node1] == 'author':
                for node2 in n.nodes:
                    if node_info['class'][node2] == 'author' and node1 != node2 and not node2 in visited_nodes:
                        intersect = n.successors[node1].intersection(n.successors[node2])
                        w_min = 0
                        w_max = 0
                        for file in intersect:
                            #print((node1, file), ':', n.edges[(node1, file)])
                            w_min = w_min + min(n.edges[(node1, file)]['weight'], n.edges[(node2, file)]['weight'])
                            w_max = w_max + max(n.edges[(node1, file)]['weight'], n.edges[(node2, file)]['weight'])
                        if w_max > 0:
                            new_n.add_edge(node1, node2, weight=w_min/w_max)
                visited_nodes.append(node1)
                
                
        return new_n
    else:
        n, node_info, edge_info = git2net.get_coauthorship_network(sqlite_db_file, time_from=min_date, time_to=max_date)
        return n

In [103]:
def pathpy_to_networkx(n):
    G = nx.Graph()
    for node in n.nodes:
        G.add_node(node)
    
    for n1, n2 in n.edges:
        G.add_edge(n1, n2, weight=n.edges[(n1, n2)]['weight'])
        
    return G

In [104]:
def network_stats_nx(G):
    d = {}

    degrees = dict(G.degree())

    clust_coeff = nx.clustering(G, weight='weight')

    deg_centrality = nx.degree_centrality(G)
    try:
        max_dc = max(deg_centrality.values())
    except ValueError:
        max_dc = 0
    sum_dc = 0    
    for node in deg_centrality:
        sum_dc = sum_dc + (max_dc - deg_centrality[node])
    
    G_n = nx.Graph()
    for node in G.nodes:
        G_n.add_node(node)
    for u,v,a in G.edges(data=True):
        G_n.add_edge(u,v, weight=a['weight']*(-1)+1)
    shortest_paths = nx.shortest_path_length(G_n, weight='weight')
    s_p = 0
    i = 0
    for start, targets in shortest_paths:
        s_p = s_p + sum(targets.values())
        i = i + len(targets.values())
    
    k_core = len([x for x in nx.core_number(G) if nx.core_number(G)[x] >= max(nx.core_number(G).values())*0.8])
    deg_c_core = len([x for x in deg_centrality if deg_centrality[x] >= max(deg_centrality.values())*0.8])
    
    x=[]
    y=[]
    for node in G.nodes:
        if not (G.degree[node] == 0 and clust_coeff[node] == 0):
            x.append(G.degree[node])
            y.append(clust_coeff[node])
    
    d['Number of nodes'] = G.number_of_nodes()
    d['Network density'] = nx.density(G)
    try:
        d['Mean degree'] = sum(degrees.values())/len(degrees.values())
    except ZeroDivisionError:
        d['Mean degree'] = 0
    if G.number_of_nodes() > 2:
        d['Degree centrality'] = sum_dc / (G.number_of_nodes() - 2)
    else:
        d['Degree centrality'] = 0
    try:
        d['Clustering coeff.'] = sum(clust_coeff.values())/len(clust_coeff.values())
    except ZeroDivisionError:
        d['Clustering coeff.'] = 0
    try:
        d['Mean path length'] = s_p / i
    except ZeroDivisionError:
        d['Mean path length'] = 0
    d['Connected components'] = nx.number_connected_components(G)
    d['K-Core count'] = k_core
    d['Degree centrality core'] = deg_c_core
    if G.number_of_nodes() > 0:
        d['Core/periphery (k-core)'] = k_core / G.number_of_nodes()
        d['Core/periphery (degree)'] = deg_c_core / G.number_of_nodes()
    else:
        d['Core/periphery (k-core)'] = 0
        d['Core/periphery (degree)'] = 0
    try:
        d['Hierarchy'] = np.polyfit(x, y, 1)[0]
    except TypeError:
        d['Hierarchy'] = 0
    
    return d

In [105]:
def generate_random_network(n, node_info):
    rnet = pp.Network(directed=True)
    files = [x for x in node_info['class'] if node_info['class'][x] == 'file' and x in n.nodes]
    #print(len(files))
    authors = [x for x in node_info['class'] if node_info['class'][x] == 'author' and x in n.nodes]
    #print(len(authors))
    
    for edge in n.edges:
        rnet.add_edge(random.choice(authors), random.choice(files))
    
    return rnet

# QUALITATIVE SECTION

In [199]:
def get_repo_details(repo):
    owner = repo.split('/')[0]
    name = repo.split('/')[1]
    
    sqlite_db_file = 'databases/{r}/{r2}.db'.format(r=repo.lower(), r2=name.lower())
    db = sqlite3.connect(sqlite_db_file)
    with open('edits_query.sql', 'r') as f:
        sql = f.read()
    edits = pd.read_sql_query(sql, db)
    
    db = sqlite3.connect('../../repo_tools/repo_tools/joci_releases_v3.db')
    with open('releases_query.sql', 'r') as f:
        sql = f.read().format(owner='\'' + owner + '\'', repo_name='\'' + name + '\'')
    releases = pd.read_sql_query(sql, db)
    
    with open('issues_query.sql', 'r') as f:
        sql = f.read().format(owner='\'' + owner + '\'', repo_name='\'' + name + '\'')
    issues = pd.read_sql_query(sql, db)
    
    
    with open('feature_keywords.json','r') as f:
        feature_keywords = json.load(f)

    with open('bug_keywords.json','r') as f:
        bug_keywords = json.load(f)

    with open('drop_words.json','r') as f:
        drop_words = json.load(f)

    issues['issue_title'] = issues['issue_title'].str.lower().str.replace('|'.join(drop_words),'', regex=True)

    issues['is_feature'] = issues['issue_title'].str.contains('|'.join(feature_keywords))
    issues['is_bug'] = issues['issue_title'].str.contains('|'.join(bug_keywords))

    issues['bug_or_feature'] = 'unknown'
    issues.loc[(issues['is_feature'] == True) & (issues['is_bug'] == False), 'bug_or_feature'] = 'feature'
    issues.loc[(issues['is_feature'] == False) & (issues['is_bug'] == True), 'bug_or_feature'] = 'bug'
    del issues['is_bug']
    del issues['is_feature']

    issues['created_at'] = pd.to_datetime(issues['created_at'], format='%Y-%m-%dT%H:%M:%SZ')
    issues['closed_at'] = pd.to_datetime(issues['closed_at'], format='%Y-%m-%dT%H:%M:%SZ')
    issues['open_for'] = issues['closed_at'] - issues['created_at']

    edits[pd.isnull(edits['lines_added']) | pd.isnull(edits['lines_removed'])] = 0 # Binary file changes have lines of NaN --> consider as 0 lines changed
    edits = edits.drop(edits[edits['hash'] == 0].index) # Sometimes the hash is 0, with all the other columns, and this leads to TypeError
    edits['author_date'] = pd.to_datetime(edits['author_date'], format='%Y-%m-%d %H:%M:%S')
    releases.sort_values(by=['created_at'])
    releases['created_at'] = pd.to_datetime(releases['created_at'], format='%Y-%m-%dT%H:%M:%SZ')
    releases['prev_created_at'] = releases.created_at.shift()
    releases['prev_tag_name'] = releases.tag_name.shift()
    
    versions = releases['tag_name'].str.extract(r'.*(\d+)\.(\d+)\.(\d+).*')
    #pd.isnull(versions)
    versions_prev = releases['prev_tag_name'].str.extract(r'.*(\d+)\.(\d+)\.(\d+).*')
    versions_prev[0] == versions[0]
    releases['release_type'] = 'unknown'
    releases.loc[(versions[0] != versions_prev[0]) & (pd.isnull(versions_prev[0]) == False) & (pd.isnull(versions[0]) == False), 'release_type'] = 'major'
    releases.loc[
        ((versions[0] == versions_prev[0]) | 
        (pd.isnull(versions_prev[0]) == True)) &
        ((versions[1] != versions_prev[1]) &
        (pd.isnull(versions_prev[1]) == False)),
        'release_type'
    ] = 'minor'
    releases.loc[
        ((versions[0] == versions_prev[0]) | 
        (pd.isnull(versions_prev[0]) == True)) & 
        ((versions[1] == versions_prev[1]) | 
        (pd.isnull(versions_prev[1]) == True)) & 
        ((versions[2] != versions_prev[2]) & 
        (pd.isnull(versions_prev[2]) == False)), 
        'release_type'
    ] = 'patch'

    r = {
        "no_of_modifications": [],
        "lines_added": [],
        "lines_removed": []
    }
    for index, row in releases.iterrows():
        for col in r:
            if pd.isnull(row['prev_created_at']):
                #print(edits['author_date'], type(edits['author_date']))
                #print(row['created_at'], type(row['created_at']))
                r[col].append(sum(edits.loc[(edits['author_date'] < row['created_at']), col]))
                continue
            r[col].append(sum(edits.loc[(edits['author_date'] >= row['prev_created_at']) & (edits['author_date'] < row['created_at']), col]))

    releases['no_of_modifications'] = r['no_of_modifications']
    releases['lines_added'] = r['lines_added']
    releases['lines_removed'] = r['lines_removed']
    releases['lines_changed'] = releases['lines_added'] + releases['lines_removed']
    
    releases = releases.iloc[1:]
    
    return issues, edits, releases

def get_cv_value(releases):
    dates = list(releases[(releases['release_type'] == 'major') | (releases['release_type'] == 'minor')]['created_at'].dt.date)
    l = []
    for i in range(1, len(dates)):
        l.append((dates[i]-dates[i-1]).days)
    try:   
        cv = stdev(l)/mean(l)
    except StatisticsError:
        cv = None
    return cv

def get_metrics_for_release(sqlite_db_file, git_repo_dir, t, node_info, duplicate_files, release, timespan, when):
    stat_names = [
        'Number of nodes', 'Network density', 'Mean degree', 'Degree centrality', 'Clustering coeff.', 'Mean path length', 
        'Connected components', 'K-Core count', 'Core/periphery (k-core)', 'Degree centrality core', 'Core/periphery (degree)',
        'Hierarchy'
    ]
    day = list(release.values())[0].date()
    if when == 'before':
        start = datetime.combine(day-timespan, datetime.min.time())
        end = datetime.combine(day, datetime.min.time())
    else:
        start = datetime.combine(day, datetime.min.time())
        end = datetime.combine(day+timespan, datetime.min.time())
    #n = collab_network(sqlite_db_file, git_repo_dir, t, node_info, day, day+timespan, True)
    n_bip = pp.Network.from_temporal_network(t, min_time=int(datetime.timestamp(start)), max_time=int(datetime.timestamp(end)))
    rstat = []
    for i in range(10):
        rnet_bip = generate_random_network(n_bip, node_info)
        rnet = collab_network_jaccard(sqlite_db_file, git_repo_dir, rnet_bip, node_info, start, end, duplicate_files, True, False)
        Gr = pathpy_to_networkx(rnet)
        s = network_stats_nx(Gr)
        rstat.append(s)

    n = collab_network_jaccard(sqlite_db_file, git_repo_dir, t, node_info, start, end, duplicate_files, True)
    G = pathpy_to_networkx(n)
    s=network_stats_nx(G)

    zstat = {}
    #print(rstat)
    for sname in stat_names:
        l = [x[sname] for x in rstat]
        r_stdev = stdev(l)
        r_mean = mean(l)
        if r_stdev != 0:
            z = (s[sname] - r_mean) / r_stdev
            zstat[sname] = z
        else:
            zstat[sname] = None
            #zstat[sname] = (s[sname] - mean(l))
    result = {
        'Metrics': s,
        'Z-values': zstat
    }
    return result

In [201]:
with open('sampling/releases_corrected_wo_error.json', 'r') as f:
    repo_releases_raw = json.load(f)
    
repo_releases = {}

for repo in repo_releases_raw:
    owner = repo.split('/')[0]
    name = repo.split('/')[1]
    db = sqlite3.connect('../../repo_tools/repo_tools/joci_releases_v3.db')
    with open('releases_query.sql', 'r') as f:
        sql = f.read().format(owner='\'' + owner + '\'', repo_name='\'' + name + '\'')
    releases = pd.read_sql_query(sql, db)
    release_names = list(releases['tag_name'])
    release_dates  = list(releases['created_at'])
    
    repo_releases[repo] = []
    for i in range(1, len(release_names)): # We skip the first release, because it gets deleted when identifying the semantic versioning
        dt = datetime.strptime(release_dates[i], '%Y-%m-%dT%H:%M:%SZ')
        repo_releases[repo].append({release_names[i]: dt})

stat_names = [
    'Number of nodes', 'Network density', 'Mean degree', 'Degree centrality', 'Clustering coeff.', 'Mean path length', 
    'Connected components', 'K-Core count', 'Core/periphery (k-core)', 'Degree centrality core', 'Core/periphery (degree)',
    'Hierarchy'
]
         
df = pd.DataFrame(columns=['repo', 'release', 'release_date', 'regularity', 'major', 'minor', 'patch', 'change_size'])
for a in ['before', 'after']:
    for sname in stat_names:
        df[sname + "_" + a] = ""

i = 1
for repo in repo_releases:
    print('Repo: ', repo, ' ({i}/{n})'.format(i=i, n=len(repo_releases)))
    l = []
    
    # Get mined data for the repo - issues, edits and releases as dataframes
    issues, edits, releases = get_repo_details(repo)
    CV = get_cv_value(releases)
    
    # Temporal network generation
    sqlite_db_file = 'databases/{r}/{r2}.db'.format(r=repo.lower(), r2=repo.split('/')[1].lower())
    git_repo_dir = 'repos/{r}4analysis'.format(r=repo.lower())
    t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file)
    # Find duplicate files from the network
    duplicate_files = duplicate_filenames(git_repo_dir)
    timespan = timedelta(days=28) # Timespan to be considered for the stats
    for release in repo_releases[repo]:
        print(release)
        metrics = {
            'before': get_metrics_for_release(sqlite_db_file, git_repo_dir, t, node_info, duplicate_files, release, timespan, 'before'),
            'after': get_metrics_for_release(sqlite_db_file, git_repo_dir, t, node_info, duplicate_files, release, timespan, 'after')
        } 
        
        release_name = list(release.keys())[0]
        #print('#1. ', releases)
        #print('#2. ', releases[releases['tag_name'] == release_name])
        release_type = list(releases[releases['tag_name'] == release_name]['release_type'])[0]
        change_size  = list(releases[releases['tag_name'] == release_name]['lines_changed'])[0]
        d = {
            'repo': repo,
            'release': list(release.keys())[0],
            'release_date': list(release.values())[0],
            'regularity': CV,
            'major': 1 if release_type=='major' else 0,
            'minor': 1 if release_type=='minor' else 0,
            'patch': 1 if release_type=='patch' else 0,
            'change_size': change_size,
        }
        for a in ['before', 'after']:
            for sname in stat_names:
                d[sname + "_" + a] = metrics[a]['Metrics'][sname]
        l.append(d)
    df = df.append(l, ignore_index = True)
    i+=1

Repo:  nccgroup/ScoutSuite  (1/81)
95
{'4.0.4': datetime.datetime(2018, 12, 11, 8, 29, 37)}
{'4.0.5': datetime.datetime(2018, 12, 12, 8, 35, 29)}
{'4.0.6': datetime.datetime(2018, 12, 22, 16, 49)}
{'4.1.0': datetime.datetime(2019, 2, 5, 15, 3, 16)}
{'4.2.0': datetime.datetime(2019, 2, 19, 13, 10, 34)}
{'4.2.post5': datetime.datetime(2019, 2, 20, 16, 3, 41)}
{'4.3.0': datetime.datetime(2019, 3, 7, 19, 33, 12)}
{'5.0.0-beta': datetime.datetime(2019, 4, 4, 13, 2, 11)}
{'5.0.0-beta2': datetime.datetime(2019, 4, 29, 17, 17, 3)}
{'5.0.post3': datetime.datetime(2019, 5, 4, 18, 35, 22)}
{'5.1.0': datetime.datetime(2019, 5, 22, 15, 1, 19)}
{'5.2.0': datetime.datetime(2019, 6, 26, 14, 31, 19)}
{'5.3.0': datetime.datetime(2019, 8, 20, 8, 48, 12)}
{'5.3.1': datetime.datetime(2019, 8, 20, 9, 15, 18)}
{'5.3.2': datetime.datetime(2019, 8, 20, 10, 44, 45)}
{'5.3.3': datetime.datetime(2019, 8, 21, 13, 58, 47)}
{'5.4.0': datetime.datetime(2019, 9, 23, 11, 12, 10)}
{'5.5.0': datetime.datetime(2019, 11, 2

{'7.0.0': datetime.datetime(2020, 2, 29, 18, 2, 37)}
{'7.0.1': datetime.datetime(2020, 3, 21, 13, 16, 40)}
{'7.0.2': datetime.datetime(2020, 4, 5, 15, 9)}
{'7.0.3': datetime.datetime(2020, 4, 18, 12, 2, 47)}
{'7.0.4': datetime.datetime(2020, 4, 29, 20, 8, 50)}
{'7.0.5': datetime.datetime(2020, 5, 15, 21, 16, 5)}
{'7.1.0': datetime.datetime(2020, 5, 31, 17, 28, 6)}
{'7.1.1': datetime.datetime(2020, 6, 21, 16, 35, 49)}
{'7.1.2': datetime.datetime(2020, 7, 4, 19, 39, 25)}
{'7.1.3': datetime.datetime(2020, 7, 17, 9, 26)}
{'7.1.4': datetime.datetime(2020, 8, 2, 15, 36, 26)}
{'7.1.5': datetime.datetime(2020, 8, 16, 17, 58, 35)}
{'7.2.0': datetime.datetime(2020, 8, 30, 20, 47, 10)}
{'7.2.1': datetime.datetime(2020, 9, 20, 19, 7, 3)}
{'7.2.2': datetime.datetime(2020, 10, 4, 11, 0, 41)}
{'7.2.3': datetime.datetime(2020, 10, 17, 21, 7, 33)}
{'7.2.4': datetime.datetime(2020, 10, 29, 8, 47, 42)}
{'7.2.5': datetime.datetime(2020, 11, 14, 18, 5, 48)}
{'7.3.0': datetime.datetime(2020, 11, 29, 20, 48,

{'v2.14.2': datetime.datetime(2017, 8, 3, 10, 58, 48)}
{'v2.15.0-beta.2': datetime.datetime(2017, 8, 3, 15, 29, 38)}
{'v2.15.0': datetime.datetime(2017, 9, 1, 10, 16, 39)}
{'v2.16.0-beta.1': datetime.datetime(2017, 9, 1, 11, 31, 36)}
{'v2.15.1': datetime.datetime(2017, 9, 6, 4, 50, 55)}
{'v2.16.0-beta.2': datetime.datetime(2017, 9, 27, 20, 18, 25)}
{'v2.16.0': datetime.datetime(2017, 10, 10, 2, 44, 22)}
{'v2.17.0-beta.1': datetime.datetime(2017, 10, 10, 3, 4, 52)}
{'v2.16.1': datetime.datetime(2017, 10, 10, 13, 58, 12)}
{'v2.16.2': datetime.datetime(2017, 10, 11, 15, 22, 56)}
{'v2.17.0-beta.2': datetime.datetime(2017, 11, 6, 20, 54, 40)}
{'v2.17.0': datetime.datetime(2017, 11, 29, 16, 26, 50)}
{'v2.18.0-beta.1': datetime.datetime(2017, 11, 29, 16, 48, 11)}
{'v2.17.1': datetime.datetime(2017, 12, 11, 18, 43, 6)}
{'v2.18.0-beta.2': datetime.datetime(2017, 12, 15, 19, 14, 52)}
{'v2.17.2': datetime.datetime(2017, 12, 27, 9, 55, 20)}
{'v2.18.0': datetime.datetime(2018, 1, 1, 18, 33, 17)}
{'

{'v7.0.7': datetime.datetime(2015, 2, 2, 4, 27, 48)}
{'v8.0.0': datetime.datetime(2015, 2, 10, 1, 48, 6)}
{'v8.0.1': datetime.datetime(2015, 2, 12, 20, 49, 2)}
{'v8.1.0': datetime.datetime(2015, 2, 25, 2, 19, 7)}
{'v8.1.1': datetime.datetime(2015, 2, 25, 5, 18, 51)}
{'v8.2.0': datetime.datetime(2015, 3, 17, 4, 37, 43)}
{'v8.3.0': datetime.datetime(2015, 4, 2, 1, 7, 36)}
{'v8.3.1': datetime.datetime(2015, 4, 5, 3, 34, 36)}
{'v8.4.0': datetime.datetime(2015, 4, 24, 21, 55, 12)}
{'v8.4.1': datetime.datetime(2015, 4, 25, 7, 51, 28)}
{'v10.0.0': datetime.datetime(2015, 6, 17, 1, 57, 23)}
{'v10.1.0': datetime.datetime(2015, 7, 6, 0, 49, 36)}
{'v10.2.0': datetime.datetime(2015, 7, 30, 6, 28, 53)}
{'v10.3.0': datetime.datetime(2015, 8, 23, 23, 55, 43)}
{'v10.3.1': datetime.datetime(2015, 8, 29, 3, 26, 47)}
{'v10.4.0': datetime.datetime(2015, 9, 17, 0, 48, 55)}
{'v10.4.1': datetime.datetime(2015, 9, 22, 8, 4, 34)}
{'v10.5.0': datetime.datetime(2015, 11, 2, 0, 31, 47)}
{'v10.6.0': datetime.datet

{'0.3.1': datetime.datetime(2020, 3, 27, 9, 43, 57)}
{'0.4.0': datetime.datetime(2020, 7, 10, 13, 35, 32)}
{'0.5.0': datetime.datetime(2021, 1, 28, 5, 41, 35)}
{'0.6.0': datetime.datetime(2021, 3, 24, 14, 52, 51)}
{'0.6.1': datetime.datetime(2021, 4, 21, 12, 3, 15)}
{'0.7.0': datetime.datetime(2021, 6, 2, 8, 43, 26)}
Repo:  nunit/nunit  (12/81)
29
{'3.0.0-alpha': datetime.datetime(2014, 9, 23, 2, 54, 54)}
{'3.0.0-alpha-2': datetime.datetime(2014, 11, 3, 4, 11, 35)}
{'3.0.0-alpha-3': datetime.datetime(2014, 11, 29, 22, 27, 26)}
{'3.0.0-alpha-4': datetime.datetime(2014, 12, 31, 0, 53, 7)}
{'3.0.0-alpha-5': datetime.datetime(2015, 1, 31, 20, 10, 19)}
{'3.0.0-beta-1': datetime.datetime(2015, 3, 26, 2, 17, 29)}
{'3.0.0-beta-2': datetime.datetime(2015, 5, 13, 0, 49, 19)}
{'3.0.0-beta-3': datetime.datetime(2015, 7, 15, 23, 18, 37)}
{'3.0.0-beta-4': datetime.datetime(2015, 8, 25, 23, 33, 16)}
{'3.0.0-beta-5': datetime.datetime(2015, 10, 17, 3, 19, 54)}
{'3.0.0-rc': datetime.datetime(2015, 11, 

...

{'v2.17.1': datetime.datetime(2019, 4, 8, 6, 42, 21)}
{'v2.17.2': datetime.datetime(2019, 4, 15, 1, 22, 5)}
{'v2.17.3': datetime.datetime(2019, 4, 16, 2, 56, 5)}
{'v2.17.4': datetime.datetime(2019, 4, 16, 21, 28, 35)}
{'v2.17.5': datetime.datetime(2019, 4, 17, 10, 31, 15)}
{'v2.17.6': datetime.datetime(2019, 4, 17, 16, 25, 49)}
{'v2.18.0': datetime.datetime(2019, 4, 17, 19, 17, 59)}
{'v2.18.2': datetime.datetime(2019, 4, 18, 16, 11, 16)}
{'v2.18.3': datetime.datetime(2019, 4, 23, 20, 34, 36)}
{'v2.18.4': datetime.datetime(2019, 4, 23, 22, 9, 37)}
{'v2.18.5': datetime.datetime(2019, 4, 24, 0, 41, 26)}
{'v2.18.6': datetime.datetime(2019, 4, 28, 4, 21, 39)}
{'v2.18.7': datetime.datetime(2019, 4, 29, 19, 38, 33)}
{'v2.18.8': datetime.datetime(2019, 4, 30, 1, 44, 32)}
{'v2.18.9': datetime.datetime(2019, 5, 1, 22, 8, 36)}
{'v2.18.10': datetime.datetime(2019, 5, 1, 23, 12, 24)}
{'v2.18.11': datetime.datetime(2019, 5, 3, 19, 54, 1)}
{'v2.18.12': datetime.datetime(2019, 5, 9, 8, 49, 51)}
{'v2.1

{'v3.0.23': datetime.datetime(2020, 3, 26, 11, 51, 56)}
{'v3.0.24': datetime.datetime(2020, 3, 29, 8, 3, 28)}
{'v3.0.25': datetime.datetime(2020, 3, 29, 9, 36, 40)}
{'v3.0.26': datetime.datetime(2020, 4, 1, 7, 53, 18)}
{'v3.0.27': datetime.datetime(2020, 4, 1, 15, 43, 16)}
{'3.0.28': datetime.datetime(2020, 4, 3, 19, 50, 26)}
{'v3.0.28': datetime.datetime(2020, 4, 3, 19, 50, 26)}
{'v3.0.29': datetime.datetime(2020, 4, 6, 2, 49, 36)}
{'v3.0.30': datetime.datetime(2020, 4, 7, 0, 31, 24)}
{'v3.0.31': datetime.datetime(2020, 4, 8, 11, 48, 2)}
{'v3.0.32': datetime.datetime(2020, 4, 13, 12, 5, 20)}
{'v3.1.0': datetime.datetime(2020, 4, 19, 8, 47, 16)}
{'v3.1.1': datetime.datetime(2020, 4, 19, 22, 54, 56)}
{'v3.1.2': datetime.datetime(2020, 4, 20, 9, 13, 31)}
{'v3.1.3': datetime.datetime(2020, 4, 20, 12, 44, 52)}
{'v3.1.4': datetime.datetime(2020, 4, 20, 22, 32, 32)}
{'v3.1.5': datetime.datetime(2020, 4, 22, 1, 49, 57)}
{'v3.1.6': datetime.datetime(2020, 4, 22, 8, 16, 56)}
{'v3.1.7': datetime

{'v3.10.3': datetime.datetime(2020, 11, 22, 21, 58, 59)}
{'p3.10.4': datetime.datetime(2020, 11, 23, 10, 10, 28)}
{'v3.10.4': datetime.datetime(2020, 11, 23, 23, 28, 33)}
{'v3.10.5': datetime.datetime(2020, 11, 25, 7, 56, 40)}
{'v3.10.6': datetime.datetime(2020, 11, 26, 14, 2, 23)}
{'v3.10.7': datetime.datetime(2020, 11, 27, 14, 20, 15)}
{'v3.10.8': datetime.datetime(2020, 11, 29, 9, 37, 59)}
{'v3.10.9': datetime.datetime(2020, 11, 30, 8, 54, 25)}
{'p3.10.9': datetime.datetime(2020, 11, 30, 18, 15, 53)}
{'v3.10.10': datetime.datetime(2020, 12, 1, 2, 23, 9)}
{'p3.10.10': datetime.datetime(2020, 12, 1, 3, 3, 5)}
{'p3.10.11': datetime.datetime(2020, 12, 1, 5, 37, 31)}
{'p3.10.12': datetime.datetime(2020, 12, 1, 6, 57, 29)}
{'p3.10.13': datetime.datetime(2020, 12, 1, 6, 59, 15)}
{'v3.11.0': datetime.datetime(2020, 12, 1, 6, 59, 15)}
{'v3.11.1': datetime.datetime(2020, 12, 1, 22, 22, 57)}
{'v3.11.2': datetime.datetime(2020, 12, 2, 12, 3, 33)}
{'v3.11.3': datetime.datetime(2020, 12, 3, 3, 57

{'8.0.0a16': datetime.datetime(2013, 9, 4, 18, 57, 18)}
{'8.0.0a17': datetime.datetime(2013, 9, 9, 19, 14, 50)}
{'8.0.0a18': datetime.datetime(2013, 9, 26, 18, 7, 8)}
{'8.0.0a19': datetime.datetime(2013, 10, 7, 22, 37, 15)}
{'8.0.0a20': datetime.datetime(2013, 10, 18, 23, 50, 58)}
{'8.0.0a21': datetime.datetime(2013, 10, 21, 17, 41, 14)}
{'8.0.0a22': datetime.datetime(2013, 10, 23, 0, 50, 54)}
{'8.0.0a23': datetime.datetime(2013, 11, 1, 21, 46, 12)}
{'8.0.0b1': datetime.datetime(2013, 11, 7, 2, 11, 33)}
{'8.0.0b2': datetime.datetime(2013, 11, 9, 0, 28, 3)}
{'8.0.0b3': datetime.datetime(2013, 11, 22, 22, 59, 54)}
{'8.0.0b4': datetime.datetime(2013, 11, 27, 15, 26, 16)}
{'8.0.0b5': datetime.datetime(2013, 12, 9, 12, 8, 16)}
{'8.0.0b6': datetime.datetime(2013, 12, 16, 19, 40, 33)}
{'8.0.0b7': datetime.datetime(2013, 12, 20, 18, 14, 39)}
{'8.0.0b8': datetime.datetime(2013, 12, 20, 18, 55, 22)}
{'8.0.0-b9': datetime.datetime(2014, 1, 21, 16, 58, 13)}
{'8.0.0': datetime.datetime(2014, 1, 29,

{'10.0.0-a39': datetime.datetime(2018, 10, 26, 18, 19, 27)}
{'10.0.0-a40': datetime.datetime(2018, 10, 30, 1, 0, 14)}
{'10.0.0-a36.3': datetime.datetime(2018, 11, 1, 20, 33, 18)}
{'10.0.0-a41': datetime.datetime(2018, 11, 13, 20, 28, 36)}
{'10.0.0-a36.4': datetime.datetime(2018, 11, 13, 20, 50, 22)}
{'10.0.0-a42': datetime.datetime(2018, 11, 29, 11, 33, 28)}
{'10.0.0-a43': datetime.datetime(2018, 11, 29, 15, 18, 11)}
{'10.0.0-a44': datetime.datetime(2018, 11, 30, 9, 13, 22)}
{'10.0.0-a36.5': datetime.datetime(2018, 11, 30, 18, 51, 53)}
{'10.0.0-a42.1': datetime.datetime(2019, 1, 2, 18, 53, 52)}
{'10.0.0-a42.2': datetime.datetime(2019, 1, 14, 18, 44, 47)}
{'10.0.0-a42.3': datetime.datetime(2019, 1, 15, 18, 50, 38)}
{'10.0.0-a45': datetime.datetime(2019, 1, 16, 16, 9, 17)}
{'10.0.0-a46': datetime.datetime(2019, 1, 29, 22, 24, 17)}
{'10.0.0-a42.4': datetime.datetime(2019, 1, 29, 23, 50, 40)}
{'10.0.0-a42.5': datetime.datetime(2019, 2, 13, 20, 7, 25)}
{'10.0.0-a47': datetime.datetime(2019,

In [136]:
df.to_csv('quantitative.csv', index=False)

In [211]:
df

,repo,release,release_date,regularity,major,minor,patch,change_size,Number of nodes_before,Network density_before,...,Mean degree_after,Degree centrality_after,Clustering coeff._after,Mean path length_after,Connected components_after,K-Core count_after,Core/periphery (k-core)_after,Degree centrality core_after,Core/periphery (degree)_after,Hierarchy_after
0,nccgroup/ScoutSuite,4.0.4,2018-12-11 08:29:37,0.503154,0,0,1,780726.0,3,0.333333,...,1.000000,0.000000,0.000000,0.166667,1,2,1.000000,2,1.000000,0.000000
1,nccgroup/ScoutSuite,4.0.5,2018-12-12 08:35:29,0.503154,0,0,1,46.0,3,0.333333,...,1.000000,0.000000,0.000000,0.166667,1,2,1.000000,2,1.000000,0.000000
2,nccgroup/ScoutSuite,4.0.6,2018-12-22 16:49:00,0.503154,0,0,1,70470.0,3,0.333333,...,2.000000,0.666667,0.283718,0.556818,1,3,0.750000,1,0.250000,0.081062
3,nccgroup/ScoutSuite,4.1.0,2019-02-05 15:03:16,0.503154,0,1,0,2708046.0,8,0.392857,...,4.857143,0.266667,0.372946,0.474384,1,6,0.857143,5,0.714286,-0.076297
4,nccgroup/ScoutSuite,4.2.0,2019-02-19 13:10:34,0.503154,0,1,0,85243130.0,8,0.535714,...,4.333333,0.200000,0.608415,0.524349,1,5,0.833333,5,0.833333,-0.071537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5197,Unidata/netcdf-c,v4.7.1,2019-08-29 17:06:26,0.692333,0,0,1,636254.0,3,0.333333,...,1.000000,0.666667,0.000000,0.266667,2,3,0.750000,1,0.250000,0.000000
5198,Unidata/netcdf-c,v4.7.2,2019-10-22 20:05:47,0.692333,0,0,1,411017.0,3,0.333333,...,1.666667,0.400000,0.158988,0.435897,2,3,0.500000,2,0.333333,0.095393
5199,Unidata/netcdf-c,v4.7.3,2019-11-21 01:15:58,0.692333,0,0,1,702122.0,8,0.178571,...,0.000000,0.000000,0.000000,0.000000,3,3,1.000000,3,1.000000,0.000000
5200,Unidata/netcdf-c,v4.7.4,2020-03-27 19:59:02,0.692333,0,0,1,60941386.0,5,0.400000,...,1.600000,0.583333,0.293982,0.117647,2,3,0.600000,1,0.200000,0.104993


In [223]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

y = np.asarray(df['change_size'], dtype=float)
X = np.asarray(df[['major', 'minor', 'patch']], dtype=int)

res_ols = sm.OLS(y, X).fit()
res_ols

In [225]:
res_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.007
Model:                            OLS   Adj. R-squared (uncentered):              0.007
Method:                 Least Squares   F-statistic:                              12.43
Date:                Sun, 06 Jun 2021   Prob (F-statistic):                    4.27e-08
Time:                        15:23:18   Log-Likelihood:                     -1.2020e+05
No. Observations:                5202   AIC:                                  2.404e+05
Df Residuals:                    5199   BIC:                                  2.404e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          7.974e+07   1.15e+08      0.695      0.487   -1.45e+08    3.05e+08
x2          3.748e+08   7.58e+07      4.944      0.000    2.26e+08    5.23e+08
x3          1.825e+08   5.19e+07      3.515      0.000    8.07e+07    2.84e+08
==============================================================================
Omnibus:                    12879.095   Durbin-Watson:                   1.490
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        159442769.682
Skew:                          26.491   Prob(JB):                         0.00
Kurtosis:                     859.037   Cond. No.                         2.21
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [226]:
res_ols.params

array([7.97389635e+07, 3.74761199e+08, 1.82523968e+08])

In [227]:
res_ols.rsquared

0.0071203985947708714